# Course work

Task: Telecom industry is a highly competitive environment. Customer aquisition is more expensinve than retention. We need to classify customers as a potential churn and try to differentiate potential churn customers by confidence level.

Metrics for predicions:
Missing potential churn is worse than making mistake about loyal customer.
*   Recall maximization
*   Precision >= 80%


Dataset for research available [here](https://www.kaggle.com/datasets/barun2104/telecom-churn)

In [1]:
import pandas as pd
import seaborn as sns
import missingno as msno
import numpy as np

print(f"pandas: {pd.__version__}")
print(f"seaborn: {sns.__version__}")
print(f"missingno: {msno.__version__}")
print(f"numpy: {np.__version__}")

pandas: 1.5.3
seaborn: 0.12.2
missingno: 0.5.2
numpy: 1.22.4


In [2]:
url = 'https://raw.githubusercontent.com/dmytrovelychko/rd_ds_python/main/datasets/cw.csv'
df = pd.read_csv(url)
print(f"shape: {df.shape}")
df[:5]

shape: (3333, 11)


,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Churn            3333 non-null   int64  
 1   AccountWeeks     3333 non-null   int64  
 2   ContractRenewal  3333 non-null   int64  
 3   DataPlan         3333 non-null   int64  
 4   DataUsage        3333 non-null   float64
 5   CustServCalls    3333 non-null   int64  
 6   DayMins          3333 non-null   float64
 7   DayCalls         3333 non-null   int64  
 8   MonthlyCharge    3333 non-null   float64
 9   OverageFee       3333 non-null   float64
 10  RoamMins         3333 non-null   float64
dtypes: float64(5), int64(6)
memory usage: 286.6 KB


In [5]:
df.dtypes

Churn                int64
AccountWeeks         int64
ContractRenewal      int64
DataPlan             int64
DataUsage          float64
CustServCalls        int64
DayMins            float64
DayCalls             int64
MonthlyCharge      float64
OverageFee         float64
RoamMins           float64
dtype: object

In [8]:
cat_cols = [
    "Churn",           # 1 if customer cancelled service, 0 if not
    "ContractRenewal", # 1 if customer recently renewed contract, 0 if not
    "DataPlan"         # 1 if customer has data plan, 0 if not
    ]
num_cols = np.setdiff1d(df.columns, cat_cols)
num_cols

array(['AccountWeeks', 'CustServCalls', 'DataUsage', 'DayCalls',
       'DayMins', 'MonthlyCharge', 'OverageFee', 'RoamMins'], dtype=object)

In [7]:
df.describe()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,0.144914,101.064806,0.903090,0.276628,0.816475,1.562856,179.775098,100.435644,56.305161,10.051488,10.237294
std,0.352067,39.822106,0.295879,0.447398,1.272668,1.315491,54.467389,20.069084,16.426032,2.535712,2.791840
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,0.000000,0.000000
25%,0.000000,74.000000,1.000000,0.000000,0.000000,1.000000,143.700000,87.000000,45.000000,8.330000,8.500000
50%,0.000000,101.000000,1.000000,0.000000,0.000000,1.000000,179.400000,101.000000,53.500000,10.070000,10.300000
75%,0.000000,127.000000,1.000000,1.000000,1.780000,2.000000,216.400000,114.000000,66.200000,11.770000,12.100000
max,1.000000,243.000000,1.000000,1.000000,5.400000,9.000000,350.800000,165.000000,111.300000,18.190000,20.000000


Churn is a target column. Other features already in a

In [ ]:
sns.pairplot(data=penguins, hue="species");

# EDA

Feature identification and their data types
* Structure Investigation
* Structure of non-numerical features
* Structure of numerical features

Statistical characteristics of the features
* Quality Investigation
* Duplicates
* Missing values

Unwanted entries and recording errors
* Content Investigation
* Feature distribution
* Feature patterns
* Feature relationships

- Validated if your data is balanced — are different groups represented in your data? Are there enough examples of each class you wish to predict?
- Is there bias in your data — are subgroups in your data treated more favorable than others?

визначити метрики якості результатів моделі

обрати і реалізувати метод валідації

натренувати модель з оптимальними гіперпараметрами

порахувати тренувальні, валідаційні і тестові метрики

зробити висновки з коротким описом, чому обрали саме ту чи іншу модель,
метрики, проаналізувати, як вдалося покращити результати протягом валідації